In [1]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2


# This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
repo_name = 'republic-project'
repo_dir = os.path.split(os.getcwd())[0].split(repo_name)[0] + repo_name
print(repo_dir)
if repo_dir not in sys.path:
    sys.path.append(repo_dir)



/Users/marijnkoolen/Code/Huygens/republic-project


In [2]:
from fuzzy_search.fuzzy_template_searcher import FuzzyTemplateSearcher, FuzzyTemplate

from republic.model.resolution_phrase_model import resolution_phrase_sets as rps
from republic.model.resolution_templates import opening_templates
from fuzzy_search.fuzzy_phrase_model import PhraseModel


searcher_config = {
    "char_match_threshold": 0.7,
    "ngram_threshold": 0.5,
    "levenshtein_threshold": 0.7,
    "max_length_variance": 3,
    "include_variants": True,
    "filter_distractors": True,
    "ngram_size": 2,
    "skip_size": 2,
}



In [6]:
[t['label'] for t in opening_templates]

['proposition_from_correspondence',
 'proposition_from_statement',
 'proposition_short_request',
 'proposition_representatives']

In [9]:
import json

print(json.dumps(opening_templates[0], indent=4))

{
    "label": "proposition_from_correspondence",
    "ordered": true,
    "type": "group",
    "elements": [
        {
            "label": "formula",
            "type": "group",
            "elements": [
                {
                    "label": "proposition_opening",
                    "required": true
                }
            ]
        },
        {
            "label": "proposer",
            "type": "group",
            "ordered": false,
            "elements": [
                {
                    "label": "person_name_prefix",
                    "required": false,
                    "cardinality": "multi"
                },
                {
                    "label": "title",
                    "required": false,
                    "cardinality": "multi"
                },
                {
                    "label": "proposer_name",
                    "required": false,
                    "variable": true
                },
                {
           

In [11]:
print(rps.keys())
opening_set_names = [
    'proposition_opening_phrases', 'person_role_phrases', 'proposition_from_phrases', 'proposition_verbs',
    'esteem_titles', 'misc', 'provinces', 'prefix_phrases', 'resolution_link_phrases',
    'location_phrases', 'organisation_phrases', 'misc', 
]
phrases = [phrase for set_name in opening_set_names for phrase in rps[set_name]]

for phrase in phrases:
    if 'Mogende' in phrase['phrase']:
        print(phrase)
phrase_model = PhraseModel(model=phrases)
opening_template = FuzzyTemplate(phrase_model, opening_templates[0])

opening_searcher = FuzzyTemplateSearcher(opening_template, searcher_config)




dict_keys(['proposition_opening_phrases', 'proposition_reason_phrases', 'proposition_closing_phrases', 'proposition_from_phrases', 'proposition_verbs', 'decision_phrases', 'resolution_link_phrases', 'prefix_phrases', 'organisation_phrases', 'location_phrases', 'esteem_titles', 'person_role_phrases', 'military_phrases', 'misc', 'provinces'])
{'phrase': 'haar Hoog Mogende', 'label': ['title', 'organisation_title'], 'variants': ['haar Hoog Mog.', 'haar Ho. Mo.']}


In [22]:
from republic.elastic.republic_elasticsearch import initialize_es
from republic.config.republic_config import set_config_inventory_num

rep_es = initialize_es(host_type="external")

inv_num = 3793
inv_config = set_config_inventory_num(inv_num, ocr_type="pagexml")



In [24]:
from republic.extraction.extract_resolution_metadata import skip_resolution, add_resolution_metadata
from republic.extraction.extract_resolution_metadata import VariableMatcher


skip_formulas = {
    'heeft aan haar Hoog Mog. voorgedragen',
    'heeft ter Vergadering gecommuniceert ',
    'ZYnde ter Vergaderinge geëxhibeert vier Pasporten van',
    'hebben ter Vergaderinge ingebraght',
    'hebben ter Vergaderinge voorgedragen'
}




In [28]:
from republic.elastic.republic_elasticsearch import initialize_es
from republic.config.republic_config import set_config_inventory_num

rep_es = initialize_es(host_type="external")

inv_num = 3793

query = {
    'query': {
        'match': {
            'metadata.inventory_num': inv_num,
        }
    }
}
resolutions = rep_es.retrieve_resolutions_by_query(query)
len(resolutions)


10

In [33]:
para_id = 'session-1750-07-13-num-1-para-3'
query = {
    'match': {
        'text_id.keyword': para_id
    }
}
response = rep_es.es_anno.search(index='phrase_matches', query=query, size=100)

matches = [hit['_source'] for hit in response['hits']['hits']]
for match in matches:
    print(f"{match['phrase']}\n\t{match['label']}")

ONtfangen een Missive van
	['proposition_opening', 'proposition_from_correspondence', 'proposition_type:missive']
Missive
	['document', 'document_type:missive']
den Heere
	person_name_prefix
haar Hoog Mogende
	['title', 'organisation_title']
Minister
	['person_role', 'domain:politics', 'minister', 'representative']
by den Ryksdag
	['organisation', 'german', 'political']
geschreeven te
	correspondence_from
Regensburgh
	['location', 'region']
Dresden
	['location', 'region']
houdende advertentie.
	['proposition_verb', 'proposition_opening_end_verb', 'proposition_body']
houdende
	['proposition_verb', 'proposition_opening_end_verb', 'proposition_body']
WAAR op geen resolutie is gevallen.
	['no_decision', 'resolution_decision']
Resolutie
	['resolution_relation', 'resolution']


In [35]:
opening_searcher.find_template_matches(matches)

AttributeError: 'dict' object has no attribute 'offset'

### Updating Phrase Match Labels in Phrase Match Index

In [132]:
import elasticsearch.helpers as helpers

label = 'esteem_title'
query = {
    'query': {
        'match': {
            'label.keyword': label
        }
    },
    'track_total_hits': True,
    'size': 1000
}






to_do = 1
while to_do > 0:
    response = es_republic.search(index='republic_phrase_matches', body=query)
    clean_docs = []
    for hit in response['hits']['hits']:
        match = hit['_source']
        match['label'] = ['title' if label == 'esteem_title' else label for label in match['label']]
        if '_score' in hit:
            del hit['_score']
        clean_docs.append(hit)
    helpers.bulk(es_republic, clean_docs)
    to_do = response['hits']['total']['value']
    print('to do:', to_do)
    es_republic.indices.refresh(index='republic_phrase_matches')
#print(clean_docs)


to do: 514989
to do: 513989
to do: 512989
to do: 511989
to do: 510989
to do: 509989
to do: 508989
to do: 507989
to do: 506989
to do: 505989
to do: 504989
to do: 503989
to do: 502989
to do: 501989
to do: 500989
to do: 499989
to do: 498989
to do: 497989
to do: 496989
to do: 495989
to do: 494989
to do: 493989
to do: 492989
to do: 491989
to do: 490989
to do: 489989
to do: 488989
to do: 487989
to do: 486989
to do: 485989
to do: 484989
to do: 483989
to do: 482989
to do: 481989
to do: 480989
to do: 479989
to do: 478989
to do: 477989
to do: 476989
to do: 475989
to do: 474989
to do: 473989
to do: 472989
to do: 471989
to do: 470989
to do: 469989
to do: 468989
to do: 467989
to do: 466989
to do: 465989
to do: 464989
to do: 463989
to do: 462989
to do: 461989
to do: 460989
to do: 459989
to do: 458989
to do: 457989
to do: 456989
to do: 455989
to do: 454989
to do: 453989
to do: 452989
to do: 451989
to do: 450989
to do: 449989
to do: 448989
to do: 447989
to do: 446989
to do: 445989
to do: 444989
to do:

(10, [])

In [145]:
from fuzzy_search.fuzzy_phrase_searcher import FuzzyPhraseSearcher

text = 'Ntfangen een Miffive van den Heere Gallieris, haar Hoogh Mogende Minifter by den Rycksdagh , gefchree ven te Regensburgh den fesden deefer loopende maand, houdendeadyertentie, WAAR àNtfangén éen Mifive van dé SccreAO dari van den Heere Graave van Gronsfeld, haar Hoogh Mogende extraOrdinaris Envoyé en Plenipotentiaris aan het Hof van fijne Majetteyt den Koningh wan PruyfTen , gefchreeven te Berlyn den feevenden deler loopende maand , houdende advertentie. WAAR op geen refolutie is gevallen: '

phrases = ['houdende', 'houdende advertentie']

searcher_config = {
    "char_match_threshold": 0.7,
    "ngram_threshold": 0.5,
    "levenshtein_threshold": 0.7,
    "max_length_variance": 3,
    "include_variants": True,
    "filter_distractors": True,
    "ngram_size": 2,
    "skip_size": 2,
}

phrase_model = PhraseModel(model=phrases)

temp_searcher = FuzzyPhraseSearcher(searcher_config)
temp_searcher.index_phrase_model(phrase_model)

temp_searcher.find_matches(text)




[PhraseMatch(phrase: "houdende advertentie", variant: "houdende advertentie",string: "houdendeadyertentie", offset: 155),
 PhraseMatch(phrase: "houdende", variant: "houdende",string: "houdende", offset: 437),
 PhraseMatch(phrase: "houdende advertentie", variant: "houdende advertentie",string: "houdende advertentie", offset: 437)]

## Adding phrase match scores for exact matches

In [9]:
import elasticsearch.helpers as helpers

phrase = 'presenteerende'
query = {
    'query': {
        'match': {
            'metadata.evidence.phrase.keyword': phrase
        }
    },
    'track_total_hits': True,
    'size': 1000
}




phrase_match_index = 'republic_phrase_matches'
resolution_index = 'pagexml_resolutions'

to_do = 1
while to_do > 0:
    response = es_republic.search(index=resolution_index, body=query)
    clean_docs = []
    for hit in response['hits']['hits']:
        match = hit['_source']
        clean_docs.append(hit)
    #helpers.bulk(es_republic, clean_docs)
    to_do = response['hits']['total']['value']
    print('to do:', to_do)
    es_republic.indices.refresh(index='republic_phrase_matches')
    break
#print(clean_docs)


to do: 17634


In [18]:
from collections import Counter

import elasticsearch.helpers as helpers

phrase = 'presenteerende'
query = {
    'query': {
        'match': {
            'metadata.evidence.phrase.keyword': phrase
        }
    },
    'track_total_hits': True,
    'size': 1000
}


count = Counter()

#response = es_republic.search(index=resolution_index, body=query)
for hit in response['hits']['hits']:
    match = hit['_source']
    evidence = match['metadata']['evidence']
    for pm in evidence:
        if pm['phrase'] != phrase:
            continue
        if pm['match_scores']['levenshtein_similarity'] and pm['match_scores']['levenshtein_similarity'] <= 0.8:
            continue
        count.update([(pm['string'], pm['match_scores']['levenshtein_similarity'])])

for string, freq in count.most_common():
    print(freq, '\t', string)

21 	 ('presenterende', 0.9285714285714286)
15 	 ('prefenteerende', 0.9285714285714286)
10 	 ('presenteerende', None)
9 	 ('representeerende', 0.875)
6 	 ('reprefenteerende', 0.8125)
3 	 ('prefenterende', 0.8571428571428572)
2 	 ('pracsenteerende', 0.8666666666666667)
2 	 ('praesenteerende', 0.9333333333333333)
2 	 ('pretendeerende', 0.8571428571428572)
2 	 ('presen. teerende', 0.875)
1 	 ('prelentecrende', 0.8571428571428572)
1 	 ('tepresenteerende', 0.875)
1 	 ('-prefenteerende', 0.8666666666666667)
1 	 ('prefentcerende', 0.8571428571428572)
1 	 ('nrelenteerende', 0.8571428571428572)
1 	 ('prefenteerénde', 0.8571428571428572)
1 	 ('pre fenteerende', 0.8666666666666667)
1 	 ('prefenteerenAe', 0.8571428571428572)
1 	 ('prefenteeren de', 0.8666666666666667)
1 	 ('prefenteerënde', 0.8571428571428572)
1 	 ('rasenteerende', 0.8571428571428572)
1 	 ('pretenteerende', 0.9285714285714286)
1 	 ('prelenteerende', 0.9285714285714286)
1 	 ('reproesenteerende', 0.8235294117647058)
1 	 (':prefenteer

In [39]:

from transformers import AutoTokenizer

pretrained_model_name = "emanjavacas/GysBERT"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name)

tokenized_input = tokenizer.encode("This is a sample text to test the tokenizer.")

print( tokenized_input )

Downloading:   0%|          | 0.00/337 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/220k [00:00<?, ?B/s]

[2, 14216, 1508, 43, 4419, 4437, 13764, 2689, 7357, 2269, 14688, 1443, 5809, 1444, 18, 3]
